In [ ]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import plotly.graph_objects as go
from scipy import stats
import yfinance as yf

df = pd.read_csv("example.csv")
df = df[df['volume'] != 0]
df.reset_index(drop=True, inplace=True)

df['EMA'] = ta.ema(df.close, length=150)
df.tail()

df=df[0:5000]


In [ ]:
def isPivot(candle, window):
    # pivot detection function

    if candle - window < 0 or candle+window >= len(df):
        return 0

    pivotHigh = 1
    pivotLow = 2
    for i in range(candle-window, candle+window+1):
        if df.iloc[candle].low > df.iloc[i].low:
            pivotLow = 0
        if df.iloc[candle].high < df.iloc[i].high:
            pivotHigh = 0
    
    if(pivotHigh and pivotLow):
        return 3
    elif pivotHigh:
        return pivotHigh
    elif  pivotLow:
        return pivotLow
    else:
        return 0

In [ ]:
window = 10
df['isPivot'] = df.apply(lambda x: isPivot(x.name, window), axis=1)

In [ ]:
def pointpos(x):
    if x['isPivot'] == 2:
        return x['low']-1e-3
    elif x['isPivot'] == 1:
        return x['high']+1e-3
    else:
        return np.nan
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [ ]:
dfp1 = df[300:500]
fig = go.Figure(data=[go.candlestick(x=dfp1.index,
                        open=dfp1['open'],             
                        high=dfp1['high'],
                        low=dfp1['low'],
                        close=dfp1['close']
                        )])

fig.add_scatter(x=dfp1.index, y=dfp1['pointpos'], mode='markers',
                marker=dict(size=5, color='MediumPurple'),
                name='pivot')
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()